Except as otherwise noted, the content of this page is licensed under the [Creative Commons Attribution 4.0 License](https://creativecommons.org/licenses/by/4.0/) (CC BY 4.0 Andrés Suárez), and code samples are licensed under the [MIT License](https://opensource.org/licenses/MIT) (Copyright 2020 Andrés Suárez). 

In [1]:
import numpy as np

In [2]:
def truss_element(E, A, ends):
    """" Generates the stiffness matrix of a bar
    
    Args:
        E (float): bar Young module [N/mm^2]
        A (float): bar transversal section [mm^2]
        ends (matrix): coordinates of the ends of the bar
        
    Returns:
        k (matrix): bar stiffness matrix [N/mm]
        
    """
    x1, y1 = ends[0,0], ends[0,1]
    x2, y2 = ends[1,0], ends[1,1]
    dx = x2-x1
    dy = y2-y1
    
    L = np.sqrt(dx**2 + dy**2)
    c = dx/L
    s = dy/L
    
    k = (E*A/L)*np.matrix([
        [c**2,   c*s,  -c**2, -c*s],
        [c*s,    s**2, -c*s,  -s**2],
        [-c**2, -c*s,   c**2,  c*s],
        [-c*s,  -s**2,  c*s,   s**2]])
    
    return k

In [3]:
def global_stiffness_matrix(Es, As, coord, dof, barc, nax):
    """" Generates the global stiffness matrix of the structure
    
    Args:
        Es (array): bars Young modulus [N/mm^2]
        As (array): bars cross sections [mm^2]
        coord (matrix): coordinates of the bars ends [mm]
        dof (int): degrees of freedom
        cbar (array): bar connectivity
        nax (array): nodal axis
        
    Returns:
        K (matrix): global stiffness matrix [N/mm]
        
    """
    K = np.zeros((dof,dof))
    for i in range(ne):
        k_local = truss_element(Es[i], As[i], coord[np.ravel(barc[i,:]),:])
        ix = np.ravel(nax[i,:])
        K[np.ix_(ix,ix)] = K[np.ix_(ix,ix)] + k_local
    return K

In [4]:
def nodal_displacement(K, F, B, R):
    """" Estimates the nodal displacements
    
    Args:
        K (matrix): global stiffness matrix [N/mm^2]
        F (array): nodal loads [N]
        B (array): nodal axis with restrictions
        R (array): fixed displacements of the restricted nodal axis [mm]
        
    Returns:
        d (array): nodal displacements [mm]
        
    """
    dof = len(F)
    d = np.zeros([dof,1])
    free = np.setxor1d(np.arange(dof), B)
    Kf = K[np.ix_(free, free)]
    Rf = F[free] - np.dot(K[np.ix_(free,B.ravel())], R)
    sol = np.dot(np.linalg.inv(Kf), Rf)
    d[B.T] = R
    d[free] = sol
    return d

![](./fig/structure1.png)

In [5]:
# geometry
ne = 3                        # number of elements
nn = 3                        # number of nodes
dof = 2*nn                    # number of degrees of freedom
coord = 1000*np.matrix('0 0; 4 0; 0 3')      # nodal coordinates
barc = np.matrix('0 1; 1 2; 0 2')            # bar connectivity
nax = np.matrix('0 1 2 3; 2 3 4 5; 0 1 4 5') # nodal axis

# materials and sections
E1 = 70e3                     # N/mm^2
E2 = 210e3                    # N/mm^2
A1 = 3e3                      # mm^2
A2 = 2e3                      # mm^2
Es = np.array([E1, E2, E1])   # size = number of bars
As = np.array([A1, A2, A1])   # size = number of bars

# loads
F = np.zeros([dof, 1])        # array of forces
P = 150000;                   # N
alpha = 60*np.pi/180          # horizontal angle in radians
F[2] = -P*np.sin(alpha)       # horizontal proyection of P in x2
F[3] = -P*np.cos(alpha)       # vertical proyection of P in y2

# restrictions
B = np.array([[0, 1, 4]]).T    # Nodal axis...
R = np.array([[0, 0, 0]]).T    # ...with no-movement restriction

In [6]:
# global stiffness matrix
K = global_stiffness_matrix(Es, As, coord, dof, barc, nax)
print(K)

[[ 52500.      0. -52500.      0.      0.      0.]
 [     0.  70000.      0.      0.      0. -70000.]
 [-52500.      0. 106260. -40320. -53760.  40320.]
 [     0.      0. -40320.  30240.  40320. -30240.]
 [     0.      0. -53760.  40320.  53760. -40320.]
 [     0. -70000.  40320. -30240. -40320. 100240.]]


In [7]:
d = nodal_displacement(K, F, B, R)
print(d.reshape(-1,2))

[[ 0.          0.        ]
 [-4.3791202  -9.39041424]
 [ 0.         -1.07142857]]


![](./fig/structure2.png)

In [8]:
# geometry
ne = 7
nn = 5
dof = 2*nn
coord = 1000*np.matrix('0 0; 3 0; 6 0; 0 3; 3 3')
barc = np.matrix('0 1; 1 2; 0 3; \
                1 3; 1 4; 2 4; \
                3 4')
nax = np.matrix('0 1 2 3; 2 3 4 5; 0 1 6 7; \
                 2 3 6 7; 2 3 8 9; 4 5 8 9; \
                 6 7 8 9')

# materials and sections
E = 1540                      # N/mm^2
A = 190                       # mm^2
Es = E*np.ones(ne)            # size = number of bars
As = A*np.ones(ne)            # size = number of bars

# loads
F = np.zeros([dof, 1])        # array of forces
P = 100;                      # N
F[5] = -P                     # horizontal proyection of P in y2

# restrictions
B = np.array([[0, 6, 7]]).T    # Nodal axis...
R = np.array([[0, 0, 0]]).T    # ...with no-movement restriction

In [9]:
# global stiffness matrix
K = global_stiffness_matrix(Es, As, coord, dof, barc, nax)
print(K.round(1))

[[ 97.5   0.  -97.5   0.    0.    0.    0.    0.    0.    0. ]
 [  0.   97.5   0.    0.    0.    0.    0.  -97.5   0.    0. ]
 [-97.5   0.  229.5 -34.5 -97.5   0.  -34.5  34.5   0.    0. ]
 [  0.    0.  -34.5 132.    0.    0.   34.5 -34.5   0.  -97.5]
 [  0.    0.  -97.5   0.  132.  -34.5   0.    0.  -34.5  34.5]
 [  0.    0.    0.    0.  -34.5  34.5   0.    0.   34.5 -34.5]
 [  0.    0.  -34.5  34.5   0.    0.  132.  -34.5 -97.5   0. ]
 [  0.  -97.5  34.5 -34.5   0.    0.  -34.5 132.    0.    0. ]
 [  0.    0.    0.    0.  -34.5  34.5 -97.5   0.  132.  -34.5]
 [  0.    0.    0.  -97.5  34.5 -34.5   0.    0.  -34.5 132. ]]


In [10]:
# noda displacement
d = nodal_displacement(K, F, B, R)
print(d.round(2).reshape(-1,2))

[[  0.     0.  ]
 [ -2.05  -4.95]
 [ -3.08 -12.98]
 [  0.     0.  ]
 [  1.03  -5.98]]
